In [11]:
import requests
import datetime
import pandas as pd
from meteostat import Point, Monthly
from sklearn.linear_model import LinearRegression
import numpy as np
from geopy.geocoders import Nominatim


In [12]:
# Tomorrow.io API Key
API_KEY = "mTgxFT8btimLVUoCY3ZkKQiwPxP6k0d6"


In [18]:
def get_lat_lon(state, district, country="India"):
    try:
        geolocator = Nominatim(user_agent="weather_forecast_app")
        query = f"{district}, {state}, {country}"
        location = geolocator.geocode(query)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"⚠️ Could not find {query}, defaulting to Delhi.")
            return 28.7041, 77.1025
    except:
        print("⚠️ Error fetching location. Using default: Delhi")
        return 28.7041, 77.1025


In [14]:
def get_forecast_tomorrowio(lat, lon, start_date, end_date):
    url = "https://api.tomorrow.io/v4/timelines"
    headers = {"apikey": API_KEY}
    params = {
        "location": f"{lat},{lon}",
        "fields": ["temperature", "precipitationIntensity", "windSpeed"],
        "startTime": start_date.isoformat(),
        "endTime": end_date.isoformat(),
        "timesteps": "1d"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None


In [19]:
state = input("Enter state name: ")
district = input("Enter district/city name: ")
month = int(input("Enter month (1-12): "))
year = int(input("Enter year (e.g., 2025): "))

# Get coordinates
lat, lon = get_lat_lon(state, district)

# Define forecast period
start_date = datetime.datetime(year, month, 1)
if month == 12:
    end_date = datetime.datetime(year + 1, 1, 1)
else:
    end_date = datetime.datetime(year, month + 1, 1)


In [20]:
state = input("Enter state name: ")
district = input("Enter district/city name: ")
month = int(input("Enter month (1-12): "))
year = int(input("Enter year (e.g., 2025): "))

# Get coordinates
lat, lon = get_lat_lon(state, district)

# Define forecast period
start_date = datetime.datetime(year, month, 1)
if month == 12:
    end_date = datetime.datetime(year + 1, 1, 1)
else:
    end_date = datetime.datetime(year, month + 1, 1)


In [21]:
forecast = get_forecast_tomorrowio(lat, lon, start_date, end_date)

if forecast and "data" in forecast and forecast["data"]["timelines"]:
    print("\n🌦 Weather Forecast from Tomorrow.io:")
    for interval in forecast["data"]["timelines"][0]["intervals"]:
        print(interval["startTime"], 
              "Temp:", interval["values"]["temperature"], "°C", 
              "Precip:", interval["values"]["precipitationIntensity"], "mm/hr",
              "Wind:", interval["values"]["windSpeed"], "m/s")
else:
    print("\n⚠️ Tomorrow.io API unavailable or returned no data. Using historical prediction...")
    prediction = predict_weather_historical(lat, lon, month, year)
    if prediction:
        print("Predicted Weather (based on historical data):")
        print(f"🌡 Avg Temperature: {prediction['predicted_temperature']} °C")
        print(f"🌧 Precipitation: {prediction['predicted_precipitation_mm']} mm")
    else:
        print("No historical data available for this region.")



⚠️ Tomorrow.io API unavailable or returned no data. Using historical prediction...
Predicted Weather (based on historical data):
🌡 Avg Temperature: 26.59 °C
🌧 Precipitation: 111.71 mm


In [27]:
import joblib

# Save the trained historical weather regression model
joblib.dump(forecast, 'historical_weather_model.pkl')


['historical_weather_model.pkl']